In [ ]:
import pandas as pd
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

# Epidemiology Model 


# Data 
- 1. COVID-19 US cases: Directly read data from JHU github repo.
- 2. Mobility Data provided by Apple for each county in the U.S.


In [ ]:
import pandas as pd
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
us_confirmed_df = pd.read_csv(url, error_bad_lines=False)

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
us_death_df = pd.read_csv(url, error_bad_lines=False)

display(us_confirmed_df.head())
display(us_death_df.head())

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
global_recover_df = pd.read_csv(url, error_bad_lines=False)
display(global_recover_df.head())


url = "https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-m50.csv"
mobility = pd.read_csv(url, error_bad_lines=False)
display(mobility.head())
us_confirmed_df = us_confirmed_df.loc[us_confirmed_df.Admin2 != "Unassigned"]
us_death_df = us_death_df.loc[us_death_df.Admin2 != "Unassigned"]


In [ ]:
us_confirmed_df.loc[~us_confirmed_df["FIPS"].isna()]

# Key variables of Epidemiology Model
- I, the number of infected, is in the `us_comfirmed`
- S, susceptible to COVID-19, is in `us_population` - `us_confirmed`, denoted as `us_susceptible`
- R, removed, is the sum of recovered and deceased. 
- N, population

## Parameters
- Beta: the infection rate
- D: number of days a patient can stay infected

## Goal
- With existing knowledge of I,S,R,and N, use gradient descent to figure out $\theta$ = ($\beta$, D)

In [ ]:
# Data Preparation
us_confirmed = us_confirmed_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key'],axis=1).sum(axis=0).values
us_death = us_death_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key',"Population"],axis=1).sum(axis=0).values
us_recovered = global_recover_df.loc[global_recover_df["Country/Region"] == "US"].drop(
    ["Province/State","Country/Region","Lat","Long"],axis= 1).values[0]
#us_removed = us_recovered+us_death
us_removed = us_recovered+us_death
#us_population = us_death_df.Population.sum()
us_population = 1e7
us_infected = us_confirmed 
us_susceptible = us_population - us_infected

#test data
us_removed = us_removed[0:100]
us_susceptible = us_susceptible[0:100]
us_infected = us_infected[0:100]

## Gradient Descent to solve for $\beta$ and $\frac{1}{D}$

Given: 
$\begin{align*}
  &\xi = \frac{1}{D} \\
  &f_s(I_n,N,S_n) = -\beta\left(\frac{I_n}{N}\right) S_n, \\
  &f_I(I_n,N,S_n) = - I\xi + \beta\left(\frac{I_n}{N}\right) S_n,  \\
  &f_R(I_n) = I_n\xi, \\
  &h=1
\end{align*}
$

Plug the above values into
$\frac{1}{N} \sum_{n=1}^N \nabla_{\theta} \left( \left({\frac{s(n+1)-s(n)}{h} - f_s(s(n), I(n), R(n);\theta)}\right)^2
+ \left({\frac{I(n+1)-I(n)}{h} - f_I(s(n), I(n), R(n;\theta)}\right)^2 + \dots \right)$

To calculate the above the term, we need to use __chain rule__ to differentiate with respect to $\beta$ and $\xi$

$\nabla_{\beta} = 2 \cdot \left(\frac{S_{n+1} - S_{n}}{h} - \left(-\beta S_n  \frac{I_n}{N}\right) \right) \cdot \left(S_n \cdot \frac{I_n}{N}\right) + 2 \cdot \left(\frac{I_{n+1} - I_{n}}{h}  - \left( -\xi_k I_n + \beta_k \frac{I_n}{N} S_n \right)\right) \cdot \left( -S_n \cdot \frac{I_n}{N}\right)$

$\nabla_{\xi} = 2 \cdot \left(\frac{I_{n+1} - I_{n}}{h}  + \left(I_n\xi_k - \beta_k \frac{I_n}{N}S_n\right) \right) \cdot \left(I_n\right) + 2 \cdot \left(\frac{R_{n+1} - R_{n}}{h} - I_n\xi_k\right) \cdot \left( -I_n\right)$



## Tuning
First initialize $\theta$ at $\beta = 0.2 $ and $\xi = 0.1$


Then, at each iteration update $\beta$ and $\xi$ according to the rules below


$\beta_{k+1} = \beta_k - h_G \partial_\beta L(\theta|s(1),\dots,s(N)),$


$
\xi_{k+1} = \xi_k - h_G \partial_\xi L(\theta|s(1),\dots,s(N)).$
 where $h_G = 0.001$
 

In [ ]:
us_confirmed_df = us_confirmed_df.loc[us_confirmed_df.Admin2 != "Unassigned"]
us_death_df = us_death_df.loc[us_death_df.Admin2 != "Unassigned"]
def calculate_gradient(s,i,r,population,beta,epsilon):
    result1 = 0 #continue adding to solve for beta
    result2 = 0 #continue adding to solve for 1/D aka epsilon
    for n in range(len(s)-1):
        result1 += 2*(s[n+1]-s[n]+beta*s[n]*(i[n]/population))*(s[n]*i[n]/population)
        result1 += 2*(i[n+1]-i[n]-beta*s[n]*(i[n]/population) + i[n]*epsilon)*(-s[n]*i[n]/population)
        
        result2 += 2*(i[n+1]-i[n]+i[n]*epsilon-beta*i[n]*s[n]/population)*(i[n])
        result2 += 2*(r[n+1]-r[n]-i[n]*epsilon)*(-i[n])
        
    return result1,result2
def calculate(s,i,r,population,learning_rate1,learning_rate2):
    beta = 0.2
    epsilon = 1/14
    
    loss = 0
    length = len(s)
    betas = []
    ds = []
    
    for itera in range(1000): # do it for 10 iterations.
        
        loss1,loss2 = calculate_gradient(s,i,r,population,beta,epsilon)
        beta_new = beta - learning_rate1* loss1/length #0.001 is the learning rate
        epsilon_new = epsilon - learning_rate2 * loss2/length
        if (beta_new == beta) & (epsilon_new == epsilon):
            print(beta_new)
            print(1/epsilon_new)
            break
        beta = beta_new
        epsilon = epsilon_new
        betas.append(beta)
        ds.append(1/epsilon)

    return betas,ds

def get_county(start,days,county_code = 1001):
    
    county_confirmed = us_confirmed_df.loc[us_confirmed_df.FIPS == county_code].drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key'],axis=1).sum(axis=0).values
    county_death = us_death_df.loc[us_death_df.FIPS == county_code].drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key',"Population"],axis=1).sum(axis=0).values
    county_population = us_death_df.loc[us_death_df.FIPS ==county_code].Population.sum()
    county_infected = county_confirmed
    county_removed = county_death
    county_susceptible = county_population - county_infected
    return county_susceptible[start:days+start],county_infected[start:start+days],county_removed[start:start+days],county_population
def get_state(start,days,state_name = "Washington"):
    
    county_confirmed = us_confirmed_df.loc[us_confirmed_df.Province_State == state_name].drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key'],axis=1).sum(axis=0).values
    county_death = us_death_df.loc[us_death_df.Province_State == state_name].drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key',"Population"],axis=1).sum(axis=0).values
    county_population = us_death_df.loc[us_death_df.Province_State ==state_name].Population.sum()
    county_infected = county_confirmed
    county_removed = county_death
    county_susceptible = county_population - county_infected
    return county_susceptible[start:start + days],county_infected[start:start + days],county_removed[start:days+start],county_population
def get_country(start,days,country_name = "US"):
    
    county_confirmed = us_confirmed_df.loc[us_confirmed_df.Country_Region == country_name].drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key'],axis=1).sum(axis=0).values
    county_death = us_death_df.loc[us_death_df.Country_Region == country_name].drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key',"Population"],axis=1).sum(axis=0).values
    county_population = us_death_df.loc[us_death_df.Country_Region ==country_name].Population.sum()
    country_recovered = global_recover_df.loc[global_recover_df["Country/Region"] == country_name].drop(
    ["Province/State","Country/Region","Lat","Long"],axis= 1).values[0]
    county_infected = county_confirmed
    county_removed = county_death + country_recovered
    county_susceptible = county_population - county_infected
    return county_susceptible[start:start+days],county_infected[start:start+days],county_removed[start:start+days],county_population

if __name__ == "__main__":
    county_code = 1005
    
    #s,i,r,p = get_county(70,1003)
    #s,i,r,p = get_state(20,40,"Washington")
    s,i,r,p = get_country(30,37)    
    learning_rate1 = 1e-3/p*60
    learning_rate2= 1e-3/p*60
    betas,ds = calculate(s,i,r,p,learning_rate1,learning_rate2)
    plt.plot(betas)
    plt.show()
    plt.plot(ds)
    plt.show()

In [ ]:
import geopandas

us = geopandas.read_file("/home/caw062/template/data/county_boundary/cb_2018_us_county_500k.shp")
us_confirmed_df["GEOID"] = us_confirmed_df["UID"].apply(lambda x:str(x)[3:])
merged = us_confirmed_df.merge(us,on="GEOID",how="left")
gdf = geopandas.GeoDataFrame(
    merged, geometry=merged["geometry"])
map1 = gdf.loc[gdf["Province_State"] == "California"].plot(column = "11/12/20",
                                                    figsize = (10,10),legend = True)
centers = gdf.centroid
points_gdf = gdf.copy()
points_gdf["geometry"] = centers
ca_points = points_gdf.loc[points_gdf["Province_State"] == "California"]
plt.title("California Confirmed Cases, Newest")
for x, y, label in zip(ca_points.geometry.x, ca_points.geometry.y, ca_points["Admin2"]):
    map1.annotate(label, xy=(x-0.5, y), xytext=(2, 2), textcoords="offset points",color = "White")

In [ ]:
import geopandas
us_confirmed_df = us_confirmed_df.loc[us_confirmed_df.Long_ < -50]
states = geopandas.read_file("/home/caw062/template/data/state_boundary/cb_2018_us_state_500k.shp")
us_confirmed_df["STATEFP"] = us_confirmed_df["UID"].apply(lambda x:str(x)[3:5])
us_confirmed_df_by_state = us_confirmed_df.groupby("STATEFP").agg({"11/12/20":"sum"})
merged = us_confirmed_df_by_state.merge(states,on="STATEFP",how="left").dropna()
merged = merged.loc[~merged["STUSPS"].isin(["HI","AK"])]
gdf = geopandas.GeoDataFrame(
    merged, geometry=merged["geometry"])

centers = gdf.centroid
points_gdf = gdf.copy()
points_gdf["geometry"] = centers

map1 = gdf.plot(column = "11/12/20",figsize = (10,10),legend = True)
plt.title("US Confirmed Cases, by State")
for x, y, label in zip(points_gdf.geometry.x, points_gdf.geometry.y, points_gdf["STUSPS"]):
    map1.annotate(label, xy=(x-0.5, y), xytext=(2, 2), textcoords="offset points",color = "White")

In [ ]:
import geopandas
us_death_df = us_death_df.loc[us_death_df.Long_ < -50]
states = geopandas.read_file("/home/caw062/template/data/state_boundary/cb_2018_us_state_500k.shp")
us_death_df["STATEFP"] = us_death_df["UID"].apply(lambda x:str(x)[3:5])
us_death_df_by_state = us_death_df.groupby("STATEFP").agg({"11/12/20":"sum"})
merged = us_death_df_by_state.merge(states,on="STATEFP",how="left").dropna()
merged = merged.loc[~merged["STUSPS"].isin(["HI","AK"])]
gdf = geopandas.GeoDataFrame(
    merged, geometry=merged["geometry"])
map3 = gdf.plot(column = "11/12/20",figsize = (10,10),legend = True)
plt.title("US Death, newest")
for x, y, label in zip(points_gdf.geometry.x, points_gdf.geometry.y, points_gdf["STUSPS"]):
    map3.annotate(label, xy=(x-0.5, y), xytext=(2, 2), textcoords="offset points",color = "White")

In [ ]:
states = geopandas.read_file("/home/caw062/template/data/state_boundary/cb_2018_us_state_500k.shp")
mobility_by_state = mobility.loc[mobility.admin_level == 1].rename({"admin1":"NAME"},axis = 1)

merged = mobility_by_state.merge(states,on="NAME",how="left")
merged = merged.loc[~merged["STUSPS"].isin(["HI","AK"])]
gdf = geopandas.GeoDataFrame(
    merged, geometry=merged["geometry"])
map4 = gdf.plot(column = "2020-11-10",figsize = (10,10),legend = True)
plt.title("US Mobility Indicator, Newest")
for x, y, label in zip(points_gdf.geometry.x, points_gdf.geometry.y, points_gdf["STUSPS"]):
    map4.annotate(label, xy=(x-0.5, y), xytext=(2, 2), textcoords="offset points",color = "White")

In [ ]:
import numpy as np
import scipy.linalg as la
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import random
fontP = FontProperties()
fontP.set_size('xx-small')

def death_multi_plot(n, df, level, start_date= '1/22/20'):
    '''
    n: number of regions
    df: Dataframe of death cases
    level: region level: admin2/ province/ state
    '''
    randomlist = []
    for i in range(0,n):
        n = random.randint(0,df.shape[0])
        randomlist.append(n)
        print(randomlist)
    for j in randomlist:
        target_j = df.loc[j,:]
        targetj_dth_info= target_j[start_date:]
        target_region = str(target_j[level])
        yj= list (targetj_dth_info)
        xj = list (targetj_dth_info.index)
        plt.plot(xj, yj, label=target_region)
    plt.xlabel('Dates')
    plt.ylabel('Number of Dath Cases')
    plt.title('Death Cases of 10 regions in '+ level+ ' level')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', prop=fontP)
    plt.figure(figsize=[20,15])
    plt.show()
    return
death_multi_plot(5,us_death_df,"Province")

In [ ]:
import nbformat as nbf

nb = nbf.v4.new_notebook()
text = """\
# My first automatic Jupyter Notebook
This is an auto-generated notebook."""

code = """\
%pylab inline
hist(normal(size=2000), bins=50);
import pandas as pd
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
us_confirmed_df = pd.read_csv(url, error_bad_lines=False)

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
us_death_df = pd.read_csv(url, error_bad_lines=False)

display(us_confirmed_df.head())
display(us_death_df.head())

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
global_recover_df = pd.read_csv(url, error_bad_lines=False)
display(global_recover_df.head())


url = "https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-m50.csv"
mobility = pd.read_csv(url, error_bad_lines=False)
display(mobility.head())
us_confirmed_df = us_confirmed_df.loc[us_confirmed_df.Admin2 != "Unassigned"]
us_death_df = us_death_df.loc[us_death_df.Admin2 != "Unassigned"]

"""

nb['cells'] = [nbf.v4.new_markdown_cell(text),
               nbf.v4.new_code_cell(code)]
fname = 'test.ipynb'

with open(fname, 'w') as f:
    nbf.write(nb, f)

In [ ]:
ca = points_gdf.loc[points_gdf.Province_State == "California"]

In [ ]:
! pip install arcpy


In [ ]:
# find out neighbors
from arcgis import *
gis1=GIS()
gis = GIS("https://ucsdonline.maps.arcgis.com/home/item.html?id=c74bf4d93f514f0296c1ffd6dda71a73", client_id='S9hCtNsAKhoTpS87')
print("Successfully logged in as: " + gis.properties.user.username)

In [ ]:
# USA COUNTIES: https://www.arcgis.com/home/item.html?id=7566e0221e5646f99ea249a197116605
counties_ = gis.content.get('7566e0221e5646f99ea249a197116605')
counties = counties_.layers[0].query().sdf


In [1]:
import pandas as pd
from collections import defaultdict
import geopandas as gpd
import numpy as np
def generate_df(nbr):  
    fips = list(nbr.FIPS)
    for i in range(len(fips)):
        if not fips[i]>0:
            fips[i]=fips[i-1]
    nbr['FIPS']=fips
    idx = []
    for i in range(nbr.shape[0]):
        center = nbr.loc[i,'FIPS'] 
        nearby = nbr.loc[i,'N-FIPS']
        if center == nearby:
            idx.append(i)
    nbr = nbr.drop(idx)
    nbr=nbr.reset_index(drop=True)
    return nbr
def get_neighbors(code,nbr):
    dic = {}
    keys = nbr.FIPS
    values=nbr['N-FIPS']
    res = defaultdict(list) 
    for i, j in zip(keys, values): 
        res[i].append(j)
    return res[code]


In [2]:
df = pd.read_csv('county_adjacency.txt', sep= '\t',encoding= 'unicode_escape', names =['Center','FIPS','NBR','N-FIPS'])
test= generate_df(df)[['FIPS','N-FIPS']]

fp = "info.shp"
data = gpd.read_file(fp)
data = data[['FIPS','LON','LAT']]
data['FIPS']=pd.to_numeric(data['FIPS'])
nei_data = data.rename(columns = {'FIPS':'N-FIPS','LAT':"N-LAT","LON":"N-LON"})

In [3]:
"""
In four directions, +x, -x, +y, -y, set center as (0,0);

"""
step1 = test.merge(data, on='FIPS', how='left')
step2 = step1.merge(nei_data, on ='N-FIPS', how='left' )
step2['LON_diff'] = step2['LON']-step2['N-LON']
step2['LAT_diff'] = step2['LAT']-step2['N-LAT']

In [9]:
get_neighbors(1001,test)

[1021, 1047, 1051, 1085, 1101]

In [4]:
step2.head()

,FIPS,N-FIPS,LON,LAT,N-LON,N-LAT,LON_diff,LAT_diff
0,1001.0,1021,-86.6428,32.5349,-86.7188,32.8479,0.0760,-0.3130
1,1001.0,1047,-86.6428,32.5349,-87.1065,32.3258,0.4637,0.2091
2,1001.0,1051,-86.6428,32.5349,-86.1492,32.5966,-0.4936,-0.0617
3,1001.0,1085,-86.6428,32.5349,-86.6501,32.1548,0.0073,0.3801
4,1001.0,1101,-86.6428,32.5349,-86.2076,32.2202,-0.4352,0.3147


In [11]:
step2.head(20)

,FIPS,N-FIPS,LON,LAT,N-LON,N-LAT,LON_diff,LAT_diff
0,1001.0,1021,-86.6428,32.5349,-86.7188,32.8479,0.0760,-0.3130
1,1001.0,1047,-86.6428,32.5349,-87.1065,32.3258,0.4637,0.2091
2,1001.0,1051,-86.6428,32.5349,-86.1492,32.5966,-0.4936,-0.0617
3,1001.0,1085,-86.6428,32.5349,-86.6501,32.1548,0.0073,0.3801
4,1001.0,1101,-86.6428,32.5349,-86.2076,32.2202,-0.4352,0.3147
5,1003.0,1025,-87.7171,30.7278,-87.8309,31.6709,0.1138,-0.9431
6,1003.0,1053,-87.7171,30.7278,-87.1616,31.1262,-0.5555,-0.3984
7,1003.0,1097,-87.7171,30.7278,-88.2018,30.7943,0.4847,-0.0665
8,1003.0,1099,-87.7171,30.7278,-87.3658,31.5706,-0.3513,-0.8428
9,1003.0,1129,-87.7171,30.7278,-88.2071,31.4077,0.4900,-0.6799


## Forecasting 
- After finding the most likely parameters, we can generate future forecasting from the parameters. 

In [ ]:
UCSDESRI123


#us_death = [0., 0., 0., 0., 1., 3., 8., 10., 13., 16., 17., 18., 23., 24., 26., 31., 39., 41., 51., 61., 73., 99., 122., 153., 209., 276., 349., 471., 599., 803., 1061., 1318., 1720., 2202., 2578., 3186., 4090.]

def sim_fun_ODE(s,i,beta, N, D, int_steps, length):
  S = np.zeros(length)
  I = np.zeros(length)
  S[0] = s[0]
  I[0] = i[0]
  dt = 1.0/int_steps
  for l in range(length-1):
    for i in range(int_steps):
      S[l] = S[l] - beta*I[l]/N*S[l]*dt
      I[l] = I[l] + (-I[l]/D + beta*I[l]/N*S[l])*dt
    S[l+1] = S[l]
    I[l+1] = I[l]
  return S, I

def sim_fun_SDE(s,i,beta, N, D, int_steps, length):
  S = np.zeros(length)
  I = np.zeros(length)
  S[0] = s[0]
  I[0] = i[0]
  dt = 1.0/int_steps
  for l in range(length-1):
    for i in range(int_steps):
      noise_matrix = np.matrix([[beta*I[l]*S[l]/N,-beta*I[l]*S[l]/N],[-beta*I[l]*S[l]/N, beta*I[l]*S[l]/N + I[l]/D]])
      normal_noise = np.matmul(la.sqrtm(noise_matrix), np.random.normal((1,2)))
      S[l] = S[l] - beta*I[l]/N*S[l]*dt + np.sqrt(dt)*normal_noise[0]
      I[l] = I[l] + (-I[l]/D + beta*I[l]/N*S[l])*dt + np.sqrt(dt)*normal_noise[1]
    S[l+1] = S[l]
    I[l+1] = I[l]
    return S, I
length = 60

beta = 0.0609266174999754
D = 145.8343055068469
N = p    # population size

int_steps = 20
plt.scatter(y=i,x=range(0,len(i),1))
S_ODE, I_ODE = sim_fun_ODE(s,i,beta, N, D, int_steps, length)
S_SDE, I_SDE = sim_fun_SDE(s,i,beta, N, D, int_steps, length)
plt.plot(I_ODE,label='ODE result')
#plt.plot(I_SDE,label='SDE result')

plt.xlabel('Time (days)')
plt.ylabel('Persons')
plt.legend()


##beta = 0.3                  # infection rate
#D = 7                   # average duration of the infection
#N = 300000000.0/10000.0     # population size


plt.show()

In [ ]:


# simulations of the ODE/SDE SIR model:
length = 150
int_steps = 20
N = 300000000.0/10000.0     # population size

# D = 5.0                    # average duration of the infection
D = 7.0*0.7                    # average duration of the infection
beta = 0.26 + 1.0/D         # infection rate
S_ODE_5, I_ODE_5 = sim_fun_ODE(s,i,beta, N, D, int_steps, length)

D = 7.0                    # average duration of the infection
beta = 0.26 + 1.0/D         # infection rate
S_ODE_7, I_ODE_7 = sim_fun_ODE(s,i,beta, N, D, int_steps, length)

# D = 10.0                    # average duration of the infection
D = 7.0*1.2                    # average duration of the infection
beta = 0.26 + 1.0/D         # infection rate
S_ODE_10, I_ODE_10 = sim_fun_ODE(s,i,beta, N, D, int_steps, length)


D = 1.0/( 1.0/(7.0*0.7) - 1.0/7.0 + 1.0/28.0 )                    # average duration of the infection
beta = 0.26 + 1.0/D         # infection rate
S_ODE_20, I_ODE_20 = sim_fun_ODE(s,i,beta, N, D, int_steps, length)

D = 28.0                    # average duration of the infection
beta = 0.26 + 1.0/D         # infection rate
S_ODE_28, I_ODE_28 = sim_fun_ODE(s,i,beta, N, D, int_steps, length)

D = 1.0/( 1.0/28.0-1.0/7.0+1.0/(7.0*1.2) )                    # average duration of the infection
beta = 0.26 + 1.0/D         # infection rate
S_ODE_30, I_ODE_30 = sim_fun_ODE(s,i,beta, N, D, int_steps, length)


fig, ax = plt.subplots()
x= np.linspace(1.0, 150.0, 150)
ax.plot(x, I_ODE_7, label='D=7')
ax.plot(x, I_ODE_28, label='D=28')
ax.scatter(range(0,len(us_death)-19,1), us_death[19:], color = 'black', marker = 'o', label='data points')
ax.fill_between(x, I_ODE_5, I_ODE_10, color='b', alpha=0.1)
ax.fill_between(x, I_ODE_20, I_ODE_30, color='r', alpha=0.1)
plt.xlabel('Time (days)', fontsize=18)
plt.ylabel('Number of Infected', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=18)
plt.show()
plt.savefig("epi_comp.eps")